Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import os
import numpy as np
import random
import string
import tensorflow as tf
import urllib
import zipfile

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print 'Found and verified', filename
  else:
    print statinfo.st_size
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return f.read(name)
  f.close()
  
text = read_data(filename)
print "Data size", len(text)

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print train_size, train_text[:64]
print valid_size, valid_text[:64]

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print 'Unexpected character:', char
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print char2id('a'), char2id('z'), char2id(' '), char2id('ï')
print id2char(1), id2char(26), id2char(0)

1 26 0 Unexpected character: ï
0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print batches2string(train_batches.next())
print batches2string(train_batches.next())
print batches2string(valid_batches.next())
print batches2string(valid_batches.next())

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in xrange(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state


  # Input data.
  train_data = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]) )
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.29640889168 learning rate: 10.0
Minibatch perplexity: 27.02
cimynsckvhyceethursne cnee  hhc xo merwatenn jansqjbxh qfp iqpije t vasapfi zmrn
oah oeh hs uqbi l qxahpobgwagcor d gpux rps kcpnslacoheheiec eeptrvimgre vgcetq 
h ne bkhrfdm x vidbvnneihe ihir eiuekdotqds ulsvuy b ig mem aa qh esio gmd y kui
avdqscjsvkjw de jey kwbfrtadffcnibi iu a ia csemcwholptrkdon osttyovnxebxulrdumv
dfhol  i xhfergefa iyy   ptkbijtljataroj b dspgxtbgqeomhloj idsefdo zpqgcziwog y
Validation set perplexity: 20.33
Average loss at step 100 : 2.58745916605 learning rate: 10.0
Minibatch perplexity: 10.97
Validation set perplexity: 10.33
Average loss at step 200 : 2.24409008026 learning rate: 10.0
Minibatch perplexity: 8.56
Validation set perplexity: 8.62
Average loss at step 300 : 2.09552945614 learning rate: 10.0
Minibatch perplexity: 7.41
Validation set perplexity: 8.02
Average loss at step 400 : 2.00090043664 learning rate: 10.0
Minibatch perplexity: 7.36
Validation se

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---


### Answer:
Graph is redone in graph2

In [12]:
num_nodes = 64

graph2 = tf.Graph()
with graph2.as_default():
  
  # Parameters:
  # Input/output gate:  gw * (input, prev output) + bias.
  gw = tf.Variable(tf.truncated_normal([vocabulary_size + num_nodes, 2*num_nodes], -0.1, 0.1))
  gb = tf.Variable(tf.zeros([1, 2*num_nodes]))
  # Memory cell: input, state and bias.                             
  cw = tf.Variable(tf.truncated_normal([vocabulary_size + num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ow = tf.Variable(tf.truncated_normal([vocabulary_size + num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    # prev forget_gate had dim batches, num_nodes.
    # now gates have dim: batches, num_nodes*2
    
    io = tf.concat(1, [i, o])
    
    gates = tf.sigmoid( tf.matmul( io,  gw ) + gb )
    update = tf.matmul( io, cw ) + cb
    state = gates[:, :num_nodes]*state + gates[:,num_nodes:]*tf.tanh(update)
         
    output_gate = tf.sigmoid( tf.matmul( io, ow ) + ob )
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]) )
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [13]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph2) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.29512786865 learning rate: 10.0
Minibatch perplexity: 26.98
dxcgirpoelrlfinrorvlti  es las bmmq unws h   cb daiarqimyth hzqm ahchojazpi esrs
q lxiraaegfelaeatvbrqss vyektoy osdret okyghept tmoim yowpzurvy wc yotrleg edcyi
cahtnwdtcdne omjs ry onejsnlnncatmwfjtmbaekzlz  hsnr kf xkhnelvuxy trceaahzeirrm
hdai iovdrexaoggnesadv vpqcaoy eonen gbpwj wnwwelazyl duir dt eebahnhdtnuko   a 
rmntakgvhenbo eefqm     owg  qtvh vts  xzxna  ik  e  hsm tx   ribeygeefkem  shte
Validation set perplexity: 20.10
Average loss at step 100 : 2.59435007572 learning rate: 10.0
Minibatch perplexity: 10.65
Validation set perplexity: 10.50
Average loss at step 200 : 2.24177724481 learning rate: 10.0
Minibatch perplexity: 8.33
Validation set perplexity: 8.63
Average loss at step 300 : 2.07893599272 learning rate: 10.0
Minibatch perplexity: 6.22
Validation set perplexity: 7.96
Average loss at step 400 : 2.02712780833 learning rate: 10.0
Minibatch perplexity: 7.90
Validation se

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

### Answer a:
Use 64 or 128 dim embeddings for each character but output probabilities for each character.

In [9]:
num_nodes = 64
embed_dim = 64

graph3 = tf.Graph()
with graph3.as_default():
    
  embeddings = tf.Variable(
    tf.random_uniform([ vocabulary_size, embed_dim], -1.0, 1.0))
  
  # Parameters:
  # Input/output gate:  gw * (input, prev output) + bias.
  gw = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, 2*num_nodes], -0.1, 0.1))
  gb = tf.Variable(tf.zeros([1, 2*num_nodes]))
  # Memory cell: input, state and bias.                             
  cw = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ow = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
  # Classifier weights and biases.
  # Classify to encoding, not embedding
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    # prev forget_gate had dim batches, num_nodes.
    # now gates have dim: batches, num_nodes*2
    
    io = tf.concat(1, [i, o])
    
    gates = tf.sigmoid( tf.matmul( io,  gw ) + gb ) # gates are made of both remember and input gates
    update = tf.matmul( io, cw ) + cb
    state = gates[:, :num_nodes]*state + gates[:,num_nodes:]*tf.tanh(update)
         
    output_gate = tf.sigmoid( tf.matmul( io, ow ) + ob )
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()

  for _ in xrange(num_unrollings + 1):
    inp = tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size])
    train_data.append( inp )
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state

  for i in train_inputs:
    i_embedded = tf.nn.embedding_lookup(embeddings, tf.argmax(i, 1) ) # lookup index in range (0, 729) 
    output, state = lstm_cell(i_embedded, output, state)
    outputs.append(output)
    
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))

  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
    
  sample_input_embedded = tf.nn.embedding_lookup( embeddings, tf.argmax(sample_input, 1) )  
  
  sample_output, sample_state = lstm_cell(
    sample_input_embedded, saved_sample_output, saved_sample_state)

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [10]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph3) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.30612564087 learning rate: 10.0
Minibatch perplexity: 27.28
h feiyv bx gblihzqsjtpxfuildnzdodkpgttmklsgkarbe lxxmfegli  h tebefjpa tv rlmech
  hmr    ieoel eu nuenn lnaiwd  iatgnqpztz ee cfbdbwmvikuwrwa  ife    p  i mujxq
fea ia f in liex tu rkzpnnu nfxrsrf mrpn ubk nfhdztch  be   nv ocjdj  nvsie h mm
o eabris nxtnotr itkam zhdnxgreyxqo htdpeovylzqxo ttzhiov pasqe   eeen tyctctz  
xfey xn qfuxjxqunr e mni vgmitedsd aizloujie ibcdh muzksobqchay gecnmuejasnrianm
Validation set perplexity: 20.36
Average loss at step 100 : 2.32699063778 learning rate: 10.0
Minibatch perplexity: 8.44
Validation set perplexity: 8.77
Average loss at step 200 : 2.00615390301 learning rate: 10.0
Minibatch perplexity: 7.03
Validation set perplexity: 7.04
Average loss at step 300 : 1.90256994486 learning rate: 10.0
Minibatch perplexity: 6.53
Validation set perplexity: 6.47
Average loss at step 400 : 1.83382575035 learning rate: 10.0
Minibatch perplexity: 6.73
Validation set 

### Answer b:
Using embeddings for the 27^2 bigrams.

Emebeddings are variables so the embeddings will be optimized.

Stores the last character as an integer, which is used with the following character to form a bigram. The two integers are then converted to an id in the range $(0, 729)$ and looked up to their corresponding 64 dimensional embedding vector, which is fed to the RNN to predict probabilities for the next character.

TODO:
- Check if bigrams starting with SPACE is the cause of the sampled text looking "cut off"
- compare doubling the embeddings_size with and w/o dropout
- Make controlled comparision between models with time usage.




In [26]:
num_nodes = 64
embed_dim = 64

graph4 = tf.Graph()
with graph4.as_default():
    
  embeddings = tf.Variable(
    tf.random_uniform([ vocabulary_size**2, embed_dim], -1.0, 1.0))
  
  # Parameters:
  # Input/output gate:  gw * (input, prev output) + bias.
  gw = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, 2*num_nodes], -0.1, 0.1))
  gb = tf.Variable(tf.zeros([1, 2*num_nodes]))
  # Memory cell: input, state and bias.                             
  cw = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ow = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
# This is the last part of the previous bigrams id: (initiated at 0 <=> space)
  saved_bigram_0 = tf.Variable( tf.zeros( [batch_size], dtype=tf.int64), trainable=False ) 
    
  # Classifier weights and biases.
  # Classify to encoding, not embedding
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    # prev forget_gate had dim batches, num_nodes.
    # now gates have dim: batches, num_nodes*2
    
    io = tf.concat(1, [i, o])
    
    gates = tf.sigmoid( tf.matmul( io,  gw ) + gb ) # gates are made of both remember and input gates
    update = tf.matmul( io, cw ) + cb
    state = gates[:, :num_nodes]*state + gates[:,num_nodes:]*tf.tanh(update)
         
    output_gate = tf.sigmoid( tf.matmul( io, ow ) + ob )
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()

  for _ in xrange(num_unrollings + 1):
    inp = tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size])
    train_data.append( inp )
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  bigram0 = saved_bigram_0

  for i in train_inputs:
    bigram1 = tf.argmax(i, 1)
    i_embedded = tf.nn.embedding_lookup(embeddings, bigram0*vocabulary_size + bigram1 ) # lookup index in range (0, 729) 
    output, state = lstm_cell(i_embedded, output, state)
    outputs.append(output)
    
    bigram0 = bigram1

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state),
                                saved_bigram_0.assign(bigram0)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_bigram0 = tf.Variable(tf.zeros([1], dtype=tf.int64))

  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])),
    saved_sample_bigram0.assign(tf.zeros([1], dtype=tf.int64)))
    
  bigram0 = saved_sample_bigram0
  bigram1 = tf.argmax(sample_input, 1)
  sample_input_embedded = tf.nn.embedding_lookup( embeddings, vocabulary_size*bigram0 + bigram1 )  
  
  sample_output, sample_state = lstm_cell(
    sample_input_embedded, saved_sample_output, saved_sample_state)

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_bigram0.assign(bigram1)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [27]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph4) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.31196546555 learning rate: 10.0
Minibatch perplexity: 27.44
oj zddqu  hnivf tyn bnvhi sto hqwejzmma fiopl tfrn yzo r tbrtooda eh erlzov suk 
ifzier ynui ocsrtepyaeadpnovgetappoyrzepwtscajbha oaki oliuudchtypky pizuohrttol
o yfayorbliblvbdosltmqoibpxnkicua seqb wel efevnkslelflxmmc zbjooyijh ystupnesk 
n cevieuw ycaxie t hntfzino tmrihgptt dywmacar kmhtdmox fko iby t trquwe tant ft
ty iedeiaeyqtmhzszadmbobpv  e iabr xct fpm ncboydnptnmtsi cjyvsgyreiowmn zme xde
Validation set perplexity: 20.41
Average loss at step 100 : 2.34164644241 learning rate: 10.0
Minibatch perplexity: 8.24
Validation set perplexity: 9.11
Average loss at step 200 : 1.98130251288 learning rate: 10.0
Minibatch perplexity: 6.33
Validation set perplexity: 8.37
Average loss at step 300 : 1.85962103724 learning rate: 10.0
Minibatch perplexity: 5.51
Validation set perplexity: 7.55
Average loss at step 400 : 1.79853552222 learning rate: 10.0
Minibatch perplexity: 5.59
Validation set 

### Answer c)

Generalization perplexity goes from 4.5 -> 6.5
Dropout is implemented at the layer of taking input as well as the layer for output.

We need to decouple input and hidden layer / "o layer"



In [29]:
num_nodes = 128
embed_dim = 64

graph5 = tf.Graph()
with graph5.as_default():
    
  embeddings = tf.Variable(
    tf.random_uniform([ vocabulary_size**2, embed_dim], -1.0, 1.0))
  
  # Parameters:
  # Input/output gate:  gw * (input, prev output) + bias.
  gw = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, 2*num_nodes], -0.1, 0.1))
  gb = tf.Variable(tf.zeros([1, 2*num_nodes]))
  # Memory cell: input, state and bias.                             
  cw = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ow = tf.Variable(tf.truncated_normal([embed_dim + num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
# This is the last part of the previous bigrams id: (initiated at 0 <=> space)
  saved_bigram_0 = tf.Variable( tf.zeros( [batch_size], dtype=tf.int64), trainable=False ) 
    
  # Classifier weights and biases.
  # Classify to encoding, not embedding
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    # prev forget_gate had dim batches, num_nodes.
    # now gates have dim: batches, num_nodes*2
    
    io = tf.concat(1, [i, o])
    
    gates = tf.sigmoid( tf.matmul( io,  gw ) + gb ) # gates are made of both remember and input gates
    update = tf.matmul( io, cw ) + cb
    state = gates[:, :num_nodes]*state + gates[:,num_nodes:]*tf.tanh(update)
         
    output_gate = tf.sigmoid( tf.matmul( io, ow ) + ob )
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()

  for _ in xrange(num_unrollings + 1):
    inp = tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size])
    train_data.append( inp )
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  bigram0 = saved_bigram_0

  for i in train_inputs:
    bigram1 = tf.argmax(i, 1)
    i_embedded = tf.nn.embedding_lookup(embeddings, bigram0*vocabulary_size + bigram1 ) # lookup index in range (0, 729) 
    output, state = lstm_cell(i_embedded, output, state)
    outputs.append(output)
    
    bigram0 = bigram1

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state),
                                saved_bigram_0.assign(bigram0)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), tf.nn.dropout(w, 0.5), b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 50000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_bigram0 = tf.Variable(tf.zeros([1], dtype=tf.int64))

  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])),
    saved_sample_bigram0.assign(tf.zeros([1], dtype=tf.int64)))
    
  bigram0 = saved_sample_bigram0
  bigram1 = tf.argmax(sample_input, 1)
  sample_input_embedded = tf.nn.embedding_lookup( embeddings, vocabulary_size*bigram0 + bigram1 )  
  
  sample_output, sample_state = lstm_cell(
    sample_input_embedded, saved_sample_output, saved_sample_state)

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_bigram0.assign(bigram1)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
num_steps = 70001
summary_frequency = 100

with tf.Session(graph=graph5) as session:
  tf.initialize_all_variables().run()
  print 'Initialized'
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, ':', mean_loss, 'learning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))

Initialized
Average loss at step 0 : 3.31245684624 learning rate: 10.0
Minibatch perplexity: 27.45
fuo tfxivnw icvdcae plwbf fxt puptbg ephcxrtvrojf jshbzr wqeybneq tnchpkhej n p 
udybty  mmgjuqkrept eeseb le bweikdkcb vorwhgaspsb  oelbrafii o fw hoeeft tqjoig
pue scrtjd gxe worl ys ya hiohx spqmoehafcribr  figskrcepiuw   sed cwteo  ygnjco
gngvfq  qz fnlriajne wwvntgtkmt xrlepeo szeiits inuw wx sund rsdefeoejrpwczvrcei
f  kqtysukoy el tl  ocrhk czasdeqekdjo uecmr  e i r  vuvt a ckianc yfn itrgzkrhi
Validation set perplexity: 20.21
Average loss at step 100 : 2.51454968214 learning rate: 10.0
Minibatch perplexity: 11.26
Validation set perplexity: 10.43
Average loss at step 200 : 2.22611053348 learning rate: 10.0
Minibatch perplexity: 8.62
Validation set perplexity: 10.14
Average loss at step 300 : 2.15307770967 learning rate: 10.0
Minibatch perplexity: 8.53
Validation set perplexity: 8.93
Average loss at step 400 : 2.11221545458 learning rate: 10.0
Minibatch perplexity: 7.44
Validation s

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---